In [9]:
# 0. import packages

import sys
import mido
import string

import time
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [10]:
# #1.1 GPU stuff

# print ("cuda: ", torch.cuda.is_available())
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# print ("current device: ", device)
# print ("count: ", torch.cuda.device_count())

# if torch.cuda.is_available():
#     print ("device name: ", torch.cuda.get_device_name(0))
#     torch.cuda.set_device(0)

In [11]:
# 1.2 load groove dataset
import math

groove_csv = pd.read_csv('groove/info.csv')
print("groove dataset:", len(groove_csv))

# get train, test, and validation sets
train_csv = []
test_csv = []
validation_csv = []

for index, row in groove_csv.iterrows():
    if str(row.audio_filename).lower() != "nan":
        split = row['split']
        if split == "train":
            train_csv.append(row)
        elif split == "test":
            test_csv.append(row)
        elif split == "validation":
            validation_csv.append(row)
        
print ("train: ", len(train_csv))
print ("test: ", len(test_csv))
print ("validation: ", len(validation_csv))

print (train_csv[0].midi_filename)

groove dataset: 1150
train:  846
test:  124
validation:  120
drummer1/session1/1_funk_80_beat_4-4.mid


In [12]:
# convert audio files into frames
from scipy.io import wavfile

i = 0
for index in train_csv:
    audio_file_path = "groove/" + index.audio_filename
    print (i, audio_file_path)
    sample_rate, raw_data = wavfile.read(audio_file_path)
    print (i, "sample rate: ", sample_rate, " raw_data: ", raw_data.shape)
    i += 1


0 groove/drummer1/session1/1_funk_80_beat_4-4.wav
0 sample rate:  44100  raw_data:  (3835150, 2)
1 groove/drummer1/session1/10_jazz-funk_116_fill_4-4.wav
1 sample rate:  44100  raw_data:  (91241, 2)
2 groove/drummer1/session1/100_neworleans-secondline_94_beat_4-4.wav
2 sample rate:  44100  raw_data:  (5967574, 2)
3 groove/drummer1/session1/101_dance-disco_120_beat_4-4.wav
3 sample rate:  44100  raw_data:  (9170686, 2)
4 groove/drummer1/session1/102_funk_95_beat_4-4.wav
4 sample rate:  44100  raw_data:  (9912461, 2)
5 groove/drummer1/session1/103_funk_95_fill_4-4.wav
5 sample rate:  44100  raw_data:  (111410, 2)
6 groove/drummer1/session1/104_funk_95_fill_4-4.wav
6 sample rate:  44100  raw_data:  (111410, 2)
7 groove/drummer1/session1/106_funk_95_fill_4-4.wav
7 sample rate:  44100  raw_data:  (222821, 2)
8 groove/drummer1/session1/107_funk_95_fill_4-4.wav
8 sample rate:  44100  raw_data:  (111410, 2)
9 groove/drummer1/session1/108_funk_95_fill_4-4.wav
9 sample rate:  44100  raw_data:  (

C:\Users\Marco\AppData\Local\Temp\ipykernel_18592\3994227384.py:8: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, raw_data = wavfile.read(audio_file_path)


80 sample rate:  44100  raw_data:  (135692, 2)
81 groove/drummer1/session1/187_reggae_78_fill_4-4.wav
81 sample rate:  44100  raw_data:  (130745, 2)
82 groove/drummer1/session1/188_reggae_78_fill_4-4.wav
82 sample rate:  44100  raw_data:  (135692, 2)
83 groove/drummer1/session1/19_jazz-funk_116_fill_4-4.wav
83 sample rate:  44100  raw_data:  (91242, 2)
84 groove/drummer1/session1/190_reggae_78_fill_4-4.wav
84 sample rate:  44100  raw_data:  (135692, 2)
85 groove/drummer1/session1/191_reggae_78_fill_4-4.wav
85 sample rate:  44100  raw_data:  (135692, 2)
86 groove/drummer1/session1/192_reggae_78_fill_4-4.wav
86 sample rate:  44100  raw_data:  (135692, 2)
87 groove/drummer1/session1/193_reggae_78_fill_4-4.wav
87 sample rate:  44100  raw_data:  (135692, 2)
88 groove/drummer1/session1/195_reggae_78_fill_4-4.wav
88 sample rate:  44100  raw_data:  (135692, 2)
89 groove/drummer1/session1/197_reggae_78_fill_4-4.wav
89 sample rate:  44100  raw_data:  (135693, 2)
90 groove/drummer1/session1/198_r

ValueError: cannot reshape array of size 10055068 into shape (3)